<a href="https://colab.research.google.com/github/malkro7/adventofcode2022/blob/main/ACI_Assignment_1_SolutionTemplate_(solved).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial and Computational Intelligence Assignment 1

## Problem solving by Uninformed & Informed Search

List only the BITS (Name) of active contributors in this assignment:
1. S R Mallikarjun Krovvidi
2. Animesh K Sahay
3. Amit Kumar
4. ___________________
5. ___________________

Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it

Coding begins here

### 1.	Define the environment in the following block

List the PEAS decription of the problem here in this markdown block

Design the agent as PSA Agent(Problem Solving Agent)
Clear Initial data structures to define the graph and variable declarations is expected
IMPORTATANT: Write distinct code block as below

In [ ]:
import math
import random
import time
import psutil

graph={}
global visited_idastar
visited_idastar = 0
num_mutations = 0
def buildGraph(input=input):

  # first row has number of nodes and edges -> directed graph
  nodes, edges = input[0].split(",")
  print(nodes, edges)
  for i in range(1,len(input)):
    d=dict()
    node1, node2, cost, hval = input[i].strip("\n").split(",")
    node1=node1.strip()
    node2=node2.strip()
    cost=int(cost.strip())
    hval=int(hval.strip())
    if (node1 not in graph):
      d[node2]=(cost,hval)
      graph[node1]=d
    else:
      d=graph.get(node1)
      d[node2]=(cost,hval)
      graph[node1]=d
  return graph

In [ ]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)
# Heuristic function: Calculate the average cost of all paths from the current node to the goal node
def heuristic(node):
    total_cost = 0
    count = 0
    for neighbor in graph.get(node, {}):
        total_cost += graph[node][neighbor][1]
        count += 1
    return total_cost / count if count != 0 else 0

# Calculate fitness of a chromosome
def fitness(chromosome):
    cost = 0
    for i in range(len(chromosome) - 1):
        node1, node2 = chromosome[i], chromosome[i + 1]
        if node1 in graph and node2 in graph[node1]:
            cost += graph[node1][node2][1]
        else:
            return math.inf  # Return infinity if path is not valid
    return cost

In [ ]:
#Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented
# Recursive search function
def search(path, cost, bound, goal):
    node = path[-1]
    f = cost + heuristic(node)
    if f > bound:
        return f, False
    if node == goal:
        return f, True
    min_cost = math.inf
    for neighbor, (actual_cost, _) in graph.get(node, {}).items():
        if neighbor not in path:
            path.append(neighbor)
            new_cost, found = search(path, cost + actual_cost, bound, goal)
            if found:
                return new_cost, True
            if new_cost < min_cost:
                min_cost = new_cost
            path.pop()
    return min_cost, False
# Recursive search function with stats printed
def search_stat(node, goal, cost, max_depth):
    if node == goal:
        return [node], 0
    if cost > max_depth:
        return None, cost
    min_cost = math.inf
    next_node = None
    for neighbor, (actual_cost, _) in graph.get(node, {}).items():
        visited_idastar += 1
        result, new_max_depth = search_stat(neighbor, goal, cost + actual_cost, max_depth)
        if result is not None and result[-1] == goal:
            return [node] + result, 0
        if new_max_depth < min_cost:
            min_cost = new_max_depth
            next_node = neighbor
    return None, min_cost

In [ ]:
#Code block : Write fucntion to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are implemented

### 2.	Definition of Algorithm 1 (Mention the Name of the algorithm here :: IDA*)

In [ ]:
# IDA* search algorithm
def ida_star(start, goal):
    bound = heuristic(start)
    path = [start]
    while True:
        cost, found = search(path, 0, bound, goal)
        if found:
            return path
        if cost == math.inf:
            return None
        bound = cost
# IDA* search algorithm with stats printed
def ida_star_stat(start, goal):
    start_time = time.time()
    max_depth = float('inf')
    while True:
        result, new_max_depth = search_stat(start, goal, 0, max_depth)
        if result is not None:
            end_time = time.time()
            print("Time taken:", end_time - start_time, "seconds")
            process = psutil.Process()
            print("Space complexity:", process.memory_info().rss / (1024 * 1024), "MB")
            return result
        if new_max_depth == math.inf:
            print("Space complexity: Unable to find solution within memory constraints.")
            return None
        max_depth = new_max_depth

### 3.	Definition of Algorithm 2 (Mention the Name of the algorithm here :: Genetic Algorithm)

In [ ]:
# Generate random chromosome
def generate_chromosome():
    return random.sample(list(graph.keys()), len(graph))

# Crossover operation
def crossover(parent1, parent2):
    crossover_point = random.randint(0, len(parent1) - 1)
    child1 = parent1[:crossover_point]
    child2 = [gene for gene in parent2 if gene not in child1]
    return child1 + child2

# Mutation operation
def mutate(chromosome):
    index1, index2 = random.sample(range(len(chromosome)), 2)
    chromosome[index1], chromosome[index2] = chromosome[index2], chromosome[index1]
    return chromosome

# Genetic algorithm
def genetic_algorithm(population_size, generations):
    population = [generate_chromosome() for _ in range(population_size)]
    for _ in range(generations):
        population = sorted(population, key=lambda x: fitness(x))
        new_population = []
        for _ in range(population_size // 2):
            parent1, parent2 = random.sample(population[:population_size // 2], 2)
            child1 = crossover(parent1, parent2)
            child2 = crossover(parent2, parent1)
            new_population.extend([mutate(child1), mutate(child2)])
        population = new_population
    return min(population, key=fitness)

### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question.

In [ ]:
#Code Block : Function & call to get inputs (start/end state)

file = "/content/sample_data/aci_193_graph_1.csv"
with open(file) as f:
    input = f.read().splitlines()
    buildGraph(input)



7 9


### 4.	Calling the search algorithms
(For bidirectional search in below sections first part can be used as per Hint provided. Under second section other combinations as per Hint or your choice of 2 algorithms can be called .As an analyst suggest suitable approximation in the comparitive analysis section)

In [ ]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))
# Test the algorithm IDA*
start_node = 'New Delhi'
goal_node = 'Chennai'
path = ida_star(start_node, goal_node)
if path:
    print("Optimal path from", start_node, "to", goal_node, ":", path)
else:
    print("No path found from", start_node, "to", goal_node)

Optimal path from New Delhi to Chennai : ['New Delhi', 'B', 'Chennai']


In [ ]:
#Invoke algorithm 2 (Should Print the solution, path, cost etc., (As mentioned in the problem))
# Test the genetic algorithm
start_node = 'New Delhi'
goal_node = 'Chennai'
solution = genetic_algorithm(population_size=100, generations=100)
print("Optimal path from", start_node, "to", goal_node, ":", solution)
print("Total cost:", fitness(solution))

Optimal path from New Delhi to Chennai : ['E', 'D', 'B', 'C', 'New Delhi', 'A']
Total cost: inf


### 5.	Comparitive Analysis

In [67]:
#Code Block : Print the Time & Space complexity of algorithm 1
# Test the algorithm
start_node = 'New Delhi'
goal_node = 'Chennai'
path = ida_star_stat(start_node, goal_node)
if path:
    print("Optimal path from", start_node, "to", goal_node, ":", path)
else:
    print("No path found from", start_node, "to", goal_node)
print("space complexity interms of nodes", visited_idastar)

UnboundLocalError: local variable 'visited_idastar' referenced before assignment

In [ ]:
#Code Block : Print the Time & Space complexity of algorithm 2

import random
import math
import time

# Graph representation
graph = {
    'New Delhi': {'A': (5, 64), 'B': (2, 60), 'C': (7, 82), 'Chennai': (10, 85)},
    'A': {'E': (1, 110)},
    'B': {'Chennai': (3, 62)},
    'C': {'D': (5, 45)},
    'E': {'Chennai': (4, 55)},
    'D': {'Chennai': (6, 32)}
}

# Generate random chromosome
def generate_chromosome():
    return random.sample(list(graph.keys()), len(graph))

# Calculate fitness of a chromosome
def fitness(chromosome):
    cost = 0
    for i in range(len(chromosome) - 1):
        node1, node2 = chromosome[i], chromosome[i + 1]
        if node1 in graph and node2 in graph[node1]:
            cost += graph[node1][node2][1]
        else:
            return math.inf  # Return infinity if path is not valid
    return cost

# Crossover operation
def crossover(parent1, parent2):
    crossover_point = random.randint(0, len(parent1) - 1)
    child1 = parent1[:crossover_point]
    child2 = [gene for gene in parent2 if gene not in child1]
    return child1 + child2

# Mutation operation
def mutate(chromosome):
    index1, index2 = random.sample(range(len(chromosome)), 2)
    chromosome[index1], chromosome[index2] = chromosome[index2], chromosome[index1]
    return chromosome

# Genetic algorithm
def genetic_algorithm(population_size, generations):
    start_time = time.time()  # Start time measurement
    population = [generate_chromosome() for _ in range(population_size)]
    for _ in range(generations):
        population = sorted(population, key=lambda x: fitness(x))
        new_population = []
        for _ in range(population_size // 2):
            parent1, parent2 = random.sample(population[:population_size // 2], 2)
            child1 = crossover(parent1, parent2)
            child2 = crossover(parent2, parent1)
            new_population.extend([mutate(child1), mutate(child2)])
        population = new_population
    end_time = time.time()  # End time measurement
    print("Time taken:", end_time - start_time, "seconds")  # Print time taken

    return min(population, key=fitness)

# Test the genetic algorithm
start_node = 'New Delhi'
goal_node = 'Chennai'
solution = genetic_algorithm(population_size=100, generations=100)
print("Optimal path from", start_node, "to", goal_node, ":", solution)
print("Total cost:", fitness(solution))

Time taken: 0.19238519668579102 seconds
Optimal path from New Delhi to Chennai : ['E', 'New Delhi', 'D', 'B', 'C', 'A']
Total cost: inf


### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section

Comparison : _______________________________________________

________________________________________________________

_________________________________________________________